In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

In [20]:
df = pd.read_csv('dr_loans.csv')
df.head()

,Unnamed: 0,profile_id,app_id,begin_date,end_date,amount,product,status,lodging_cnt,repayment_ratio,bad,can_be_bad,loan_num
0,1,14,219,2018-04-16,2018-05-14,300,START,CLOSED,0,1.12,False,True,1
1,2,30,322,2018-04-17,2018-05-02,1100,START,CLOSED,0,1.09,False,True,1
2,3,16,304,2018-04-17,2018-04-22,500,START,CLOSED,0,1.09,False,True,1
3,4,54,334,2018-04-18,2018-04-23,300,START,CLOSED,0,1.09,False,True,1
4,5,66,344,2018-04-18,2018-04-19,400,START,CLOSED,0,1.09,False,True,1


In [21]:
# check distribution
df.groupby('loan_num').size()

loan_num
1     8743
2     2849
3     1146
4      538
5      281
6      137
7       79
8       54
9       28
10      18
11      13
12      10
13       8
14       7
15       4
16       3
17       3
18       3
19       3
20       2
21       1
22       1
23       1
24       1
dtype: int64

In [24]:
# set date format
date_format = "%Y-%m-%d"
# example of calculate mean of dates
var1 = df[(df['loan_num']==20)]['begin_date']
var2 = df[(df['loan_num']==20)]['end_date']

begin_mean = (np.array(var1, dtype='datetime64[s]')
        .view('i8')
        .mean()
        .astype('datetime64[s]'))

end_mean = (np.array(var2, dtype='datetime64[s]')
        .view('i8')
        .mean()
        .astype('datetime64[s]'))

a = datetime.strptime(str(begin_mean)[:10], date_format)
b = datetime.strptime(str(end_mean)[:10], date_format)

delta = b - a
print (delta.days)

24


In [25]:
# records to check algorithm
df[((df['loan_num']==20))]

,Unnamed: 0,profile_id,app_id,begin_date,end_date,amount,product,status,lodging_cnt,repayment_ratio,bad,can_be_bad,loan_num
10135,10136,232,623018,2019-08-25,2019-09-28,2500,START_11,CLOSED,1,1.11664,False,True,20
12509,12510,753344,759827,2019-11-26,2019-12-10,1000,START_11,CLOSED,0,1.01800,False,True,20


In [26]:
# create new empty columns for calculating
df['d_to_end'] = ''
df['d_from_prev'] = ''

# loop with date calculating
for i in range(0, len(df)):
    
    # set 
    begin = df['begin_date'].iloc[i-1]
    end = df['end_date'].iloc[i-1]

    # convert to "%Y-%m-%d" format
    converted_begin = datetime.strptime(begin, date_format)
    converted_end = datetime.strptime(end, date_format)

    # set day to end value
    day_to_end = converted_end - converted_begin   
    df['d_to_end'].iloc[i-1] = day_to_end.days
    
    # calculate count of day from previous loan of persone
    if(df['loan_num'].iloc[i-1]>1):
        profile_id = df['profile_id'].iloc[i-1]
        num = df['loan_num'].iloc[i-1]
        
        # get params of loan`s date 
        current_date = df['begin_date'].iloc[i-1]
        previous_date = df[(df['profile_id'] == profile_id) & (df['loan_num']==num-1)]['begin_date'].values[0]
       
        converted_begin = datetime.strptime(previous_date, date_format)
        converted_end = datetime.strptime(current_date, date_format)

        day_to_end = converted_end - converted_begin   
        df['d_from_prev'].iloc[i-1] = day_to_end.days
    else:
        df['d_from_prev'].iloc[i-1] = -1
    
    i += 1
df.head()

C:\Users\maks1\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Unnamed: 0,profile_id,app_id,begin_date,end_date,amount,product,status,lodging_cnt,repayment_ratio,bad,can_be_bad,loan_num,d_to_end,d_from_prev
0,1,14,219,2018-04-16,2018-05-14,300,START,CLOSED,0,1.12,False,True,1,28,-1
1,2,30,322,2018-04-17,2018-05-02,1100,START,CLOSED,0,1.09,False,True,1,15,-1
2,3,16,304,2018-04-17,2018-04-22,500,START,CLOSED,0,1.09,False,True,1,5,-1
3,4,54,334,2018-04-18,2018-04-23,300,START,CLOSED,0,1.09,False,True,1,5,-1
4,5,66,344,2018-04-18,2018-04-19,400,START,CLOSED,0,1.09,False,True,1,1,-1


In [27]:
binning_df = pd.DataFrame.copy(df[['d_to_end', 'status']])

# binning data by amount
cut_labels_4 = ['1 week','2 weeks','3+ weeks']
cut_bins = [0, 7, 14, 1000]
binning_df['d_to_end'] = pd.cut(binning_df['d_to_end'], bins=cut_bins, labels=cut_labels_4)

binning_df.head()

,d_to_end,status
0,3+ weeks,CLOSED
1,3+ weeks,CLOSED
2,1 week,CLOSED
3,1 week,CLOSED
4,1 week,CLOSED


In [28]:
# create two dataframe to calculate percentage
d1 = pd.DataFrame.copy(binning_df[binning_df['status']=='CLOSED'].groupby('d_to_end')
         .agg(closed_persent=pd.NamedAgg(column='status', aggfunc='count'))
         .reset_index()
)

d2 = pd.DataFrame.copy(binning_df.groupby('d_to_end')
         .agg(closed_persent=pd.NamedAgg(column='status', aggfunc='count'))
         .reset_index()
)

d1['closed_persent'] = 100/d2['closed_persent']*d1['closed_persent']
d1

,d_to_end,closed_persent
0,1 week,83.994126
1,2 weeks,70.871385
2,3+ weeks,65.170085
